# Performing differential gene expression with MEMENTO and  Cell Census data

The Cell Census is a versioned container for the single-cell data hosted at [CELLxGENE Discover](https://cellxgene.cziscience.com/). The Cell Census utilizes [SOMA](https://github.com/single-cell-data/SOMA/blob/main/abstract_specification.md) powered by [TileDB](https://tiledb.com/products/tiledb-embedded) for storing, accessing, and efficiently filtering data.

TBD

# Contents
TBD

## Opening the census

The `cell_census` python package contains a convenient API to open the latest version of the Cell Census.

In [1]:
import cell_census
import numpy as np
import memento
import pandas as pd

census = cell_census.open_soma()

You can learn more about the `cell_census` methods by accessing their corresponding documentation via `help()`. For example `help(cell_census.open_soma)`.

## Load data

Getting dataset of interest

In [2]:
# Tabula muris senis 10x liver
dataset_title = "All major cell types in adult human retina"

# Getting the joinid and dataset id
datasets = census["census_info"]["datasets"].read(value_filter = f"dataset_title == '{dataset_title}'").concat().to_pandas()
dataset_joinid = list(datasets["soma_joinid"])
datasset_interest_id = datasets["dataset_id"][0]

Get measured genes in dataset

In [3]:
presence_matrix = cell_census.get_presence_matrix(census, organism="Homo sapiens", measurement_name="RNA")
dataset_presnence = presence_matrix[dataset_joinid,]

gene_joinid = np.nonzero(dataset_presnence.sum(axis=0).A1 == dataset_presnence.shape[0])[0].tolist()

Get anndata

In [4]:
assays = ["10x 3' v3", "10x 3' v2"]
adata = cell_census.get_anndata(
    census=census,
    organism="Homo sapiens",
    var_coords= gene_joinid,
    #obs_value_filter=f"dataset_id == '{datasset_interest_id}'",
    obs_value_filter=f"tissue_general == 'eye' and assay in {assays}",
)

In [5]:
adata.var_names = adata.var["feature_name"]
adata.var

,soma_joinid,feature_id,feature_name,feature_length
feature_name,,,,
RP11-34P13.7,0,ENSG00000238009,RP11-34P13.7,3726
AP006222.1,2,ENSG00000228463,AP006222.1,8224
RP4-669L17.4,3,ENSG00000237094,RP4-669L17.4,6204
RP11-206L10.17,4,ENSG00000230021,RP11-206L10.17,5495
LINC01409,5,ENSG00000237491,LINC01409,8413
...,...,...,...,...
RP11-398A8.4,52844,ENSG00000273461,RP11-398A8.4,496
LL09NC01-139C3.1,52846,ENSG00000273473,LL09NC01-139C3.1,572
RP11-108L7.14,52847,ENSG00000273476,RP11-108L7.14,448


In [6]:
aaa = adata.obs["cell_type"].value_counts()
list(zip(list(aaa.index), list(aaa)))

[('retinal rod cell', 151112),
 ('amacrine cell', 122396),
 ('retinal bipolar neuron', 56261),
 ('Mueller cell', 50545),
 ('ON-bipolar cell', 46059),
 ('OFF-bipolar cell', 41015),
 ('fibroblast', 38487),
 ('corneal epithelial cell', 31410),
 ('retinal ganglion cell', 30729),
 ('retinal cone cell', 28626),
 ('melanocyte', 27699),
 ('pigmented epithelial cell', 27000),
 ('ciliary muscle cell', 21468),
 ('retina horizontal cell', 19603),
 ('glial cell', 18011),
 ('basal cell', 17465),
 ('pigmented ciliary epithelial cell', 16178),
 ('conjunctival epithelial cell', 14439),
 ('Schwann cell', 10765),
 ('corneal endothelial cell', 8394),
 ('macrophage', 7191),
 ('non-pigmented ciliary epithelial cell', 6327),
 ('anterior lens cell', 6160),
 ('glycinergic neuron', 6065),
 ('macroglial cell', 5276),
 ('lens epithelial cell', 5276),
 ('rod bipolar cell', 5207),
 ('endothelial cell', 4872),
 ('native cell', 3704),
 ('smooth muscle cell of sphincter of pupil', 2221),
 ('astrocyte', 2031),
 ('endot

## Differential gene expression: Retinal Rod Cells vs rest  

In [7]:
cell_type_interest = "retinal rod cell"
adata.obs["differential_grouping"] = adata.obs.cell_type == cell_type_interest
adata.obs["differential_grouping"] = adata.obs["differential_grouping"] * 1 

Run memento

In [28]:
differential_exp_results = memento.binary_test_1d(
    adata=adata, 
    capture_rate=0.07, 
    treatment_col='differential_grouping', 
    num_cpus=90,
    num_boot=5000
)

[Parallel(n_jobs=90)]: Using backend LokyBackend with 90 concurrent workers.
[Parallel(n_jobs=90)]: Done  21 tasks      | elapsed:   13.6s
[Parallel(n_jobs=90)]: Done 271 tasks      | elapsed:   16.4s
[Parallel(n_jobs=90)]: Done 621 tasks      | elapsed:   20.2s
[Parallel(n_jobs=90)]: Done 1071 tasks      | elapsed:   25.0s
[Parallel(n_jobs=90)]: Done 1621 tasks      | elapsed:   31.2s
[Parallel(n_jobs=90)]: Done 2271 tasks      | elapsed:   38.2s
[Parallel(n_jobs=90)]: Done 3021 tasks      | elapsed:   46.4s
[Parallel(n_jobs=90)]: Done 3871 tasks      | elapsed:   55.8s
[Parallel(n_jobs=90)]: Done 4821 tasks      | elapsed:  1.1min
[Parallel(n_jobs=90)]: Done 5871 tasks      | elapsed:  1.3min
[Parallel(n_jobs=90)]: Done 6866 out of 6866 | elapsed:  1.5min finished


In [29]:
differential_exp_results.query('de_coef > 0').sort_values('de_pval').head(10)

,gene,tx,de_coef,de_se,de_pval,dv_coef,dv_se,dv_pval
1808,FSTL5,differential_grouping,0.258739,0.003789,3.222062e-117,-0.076154,0.006957,1.555925e-03
314,ABCA4,differential_grouping,2.629044,0.004722,3.848001e-23,-2.009847,0.006603,2.448909e-06
3467,PBX3,differential_grouping,0.878122,0.004367,1.267126e-17,-0.466352,0.014185,2.423038e-09
5127,SCAMP5,differential_grouping,1.608120,0.003642,7.137868e-17,-0.073244,0.009946,9.916017e-01
4941,TECPR2,differential_grouping,0.856125,0.005099,1.460205e-16,0.201101,0.014748,2.874791e-04
1839,LRP2BP,differential_grouping,0.564324,0.005321,1.562009e-16,0.358604,0.016889,1.431961e-06
2207,RP11-69L16.4,differential_grouping,0.615859,0.007842,3.100638e-15,0.095469,0.026947,3.599280e-03
980,PRKRA,differential_grouping,0.634701,0.004727,5.897490e-15,0.097925,0.020410,1.388858e-03
4327,ZNF385A,differential_grouping,0.488646,0.005237,6.094489e-15,-0.085893,0.015123,1.000000e+00
4874,VASH1,differential_grouping,2.048481,0.006372,1.452393e-14,-1.310290,0.017718,1.293770e-07


Do it again for reproducibility check

In [31]:
differential_exp_results = memento.binary_test_1d(
    adata=adata, 
    capture_rate=0.07, 
    treatment_col='differential_grouping', 
    num_cpus=90,
    num_boot=5000
)
differential_exp_results.query('de_coef > 0').sort_values('de_pval').head(10)

[Parallel(n_jobs=90)]: Using backend LokyBackend with 90 concurrent workers.
[Parallel(n_jobs=90)]: Done  21 tasks      | elapsed:   16.6s
[Parallel(n_jobs=90)]: Done 271 tasks      | elapsed:   19.8s
[Parallel(n_jobs=90)]: Done 621 tasks      | elapsed:   23.5s
[Parallel(n_jobs=90)]: Done 1071 tasks      | elapsed:   28.3s
[Parallel(n_jobs=90)]: Done 1621 tasks      | elapsed:   34.2s
[Parallel(n_jobs=90)]: Done 2271 tasks      | elapsed:   41.3s
[Parallel(n_jobs=90)]: Done 3021 tasks      | elapsed:   49.2s
[Parallel(n_jobs=90)]: Done 3871 tasks      | elapsed:   58.5s
[Parallel(n_jobs=90)]: Done 4821 tasks      | elapsed:  1.1min
[Parallel(n_jobs=90)]: Done 5871 tasks      | elapsed:  1.3min
[Parallel(n_jobs=90)]: Done 6866 out of 6866 | elapsed:  1.5min finished


,gene,tx,de_coef,de_se,de_pval,dv_coef,dv_se,dv_pval
4811,ESR2,differential_grouping,1.545427,0.005121,1.128766e-20,-0.733870,0.042790,6.656760e-07
1615,TBC1D19,differential_grouping,0.100008,0.003893,5.471630e-17,0.621873,0.011880,1.778336e-08
38,SLC25A33,differential_grouping,0.151069,0.006078,6.261642e-17,-0.488245,0.024377,3.024781e-07
2039,LMNB1,differential_grouping,1.432111,0.007821,8.291231e-17,-0.599652,0.038441,5.824325e-06
346,KCNC4,differential_grouping,1.002515,0.008260,2.577603e-16,-0.415979,0.024480,1.346749e-05
4686,GRTP1,differential_grouping,1.475086,0.006605,4.087977e-16,-1.042588,0.022227,5.153478e-07
6788,LINC02275,differential_grouping,1.444961,0.008265,1.586436e-15,-1.504364,0.022650,4.197677e-11
4501,BICDL1,differential_grouping,0.999700,0.004311,3.838814e-15,-0.259075,0.010333,3.835746e-07
106,HMGN2,differential_grouping,0.869007,0.004520,5.623757e-15,0.726534,0.012962,2.895712e-07
6595,HMGN1,differential_grouping,1.957772,0.003568,1.066772e-14,-1.509314,0.024422,8.920429e-08


## Differential gene expression: iterate for each cell type vs rest

Now let's do it for all cell types

In [ ]:
differential_exp_results_all = pd.DataFrame()
for cell_type_interest in adata.obs.cell_type.drop_duplicates():
    
    print(cell_type_interest)
    
    # Set groupings
    adata.obs["differential_grouping"] = adata.obs.cell_type == cell_type_interest
    adata.obs["differential_grouping"] = adata.obs["differential_grouping"] * 1
    
    # Run differential gene expression
    differential_exp_results = memento.binary_test_1d(
        adata=adata, 
        capture_rate=0.07, 
        treatment_col='differential_grouping', 
        num_cpus=90,
        num_boot=5000
    )
    
    # Retain columns of interest and rename them for future joining
    differential_exp_results = differential_exp_results[["gene", "de_coef", "de_pval", "de_se"]]
    differential_exp_results.set_index("gene", inplace = True)
    differential_exp_results.rename(inplace = True, 
                                    columns = "de_coef":cell_type_interest + "de_coef", 
                                    "de_pval": cell_type_interest + "de_pval",
                                    "de_se": cell_type_interest + "de_se"
                                   )
    
    # Join to var dataframe
    adata = adata.var.merge(differential_exp_results, how = "left")
    
    
    #genes = differential_exp_results["gene"]
    
    #adata.var[cell_type_interest + "de_coef"] = 0.0
    #adata.var[cell_type_interest + "de_se"] = 100.0
    #adata.var[cell_type_interest + "de_pval"] = 1.0
    
    #adata.var.loc[genes,cell_type_interest + "de_coef"] = list(differential_exp_results["de_coef"])
    #adata.var.loc[genes,cell_type_interest + "de_se"] = list(differential_exp_results["de_se"])
    #adata.var.loc[genes,cell_type_interest + "de_pval"] = list(differential_exp_results["de_pval"])


In [ ]:
test = adata.var.query('`retinal rod cellde_coef` > 0').sort_values('retinal rod cellde_pval')

test[["retinal rod cellde_coef", "retinal rod cellde_pval"]].loc["R3HCC1",]

## Find Marker Genes (SCRAN-inspired) for Retinal Rod Cells

Let's do Retinal Rod Cells vs each other cell type. First perform all comparions across other cell types.

In [8]:
retinal_rod_markers = pd.DataFrame()
cell_type_query = "retinal rod cell"
for cell_type_target in adata.obs.cell_type.drop_duplicates():
    
    if cell_type_target == cell_type_query:
        continue
    
    print(cell_type_target)
    
    # Set groupings
    adata.obs[cell_type_target] = adata.obs.cell_type == cell_type_query
    adata.obs[cell_type_target] = adata.obs[cell_type_target] * 1
    
    # Run differential gene expression
    differential_exp_results = memento.binary_test_1d(
        adata=adata[adata.obs.cell_type.isin([cell_type_target, cell_type_query]),], 
        capture_rate=0.07, 
        treatment_col=cell_type_target, 
        num_cpus=90,
        num_boot=5000
    )
    
    # Retain columns of interest and rename them for future joining
    differential_exp_results = differential_exp_results[["gene", "de_coef", "de_pval", "de_se"]]
    
    if retinal_rod_markers.shape == (0,0):
        retinal_rod_markers = differential_exp_results
    else:
        retinal_rod_markers = retinal_rod_markers.append(differential_exp_results, ignore_index=True)

retina horizontal cell


[Parallel(n_jobs=90)]: Using backend LokyBackend with 90 concurrent workers.
[Parallel(n_jobs=90)]: Done  20 tasks      | elapsed:    9.4s
[Parallel(n_jobs=90)]: Done 270 tasks      | elapsed:   11.4s
[Parallel(n_jobs=90)]: Done 620 tasks      | elapsed:   13.7s
[Parallel(n_jobs=90)]: Done 1070 tasks      | elapsed:   16.9s
[Parallel(n_jobs=90)]: Done 1620 tasks      | elapsed:   20.9s
[Parallel(n_jobs=90)]: Done 2270 tasks      | elapsed:   25.3s
[Parallel(n_jobs=90)]: Done 3020 tasks      | elapsed:   30.4s
[Parallel(n_jobs=90)]: Done 3870 tasks      | elapsed:   36.2s
[Parallel(n_jobs=90)]: Done 4820 tasks      | elapsed:   42.7s
[Parallel(n_jobs=90)]: Done 5870 tasks      | elapsed:   49.7s
[Parallel(n_jobs=90)]: Done 6239 out of 6239 | elapsed:   52.6s finished


ON-bipolar cell


[Parallel(n_jobs=90)]: Using backend LokyBackend with 90 concurrent workers.
[Parallel(n_jobs=90)]: Done  20 tasks      | elapsed:    0.5s
[Parallel(n_jobs=90)]: Done 360 tasks      | elapsed:    3.5s
[Parallel(n_jobs=90)]: Done 889 tasks      | elapsed:    7.2s
[Parallel(n_jobs=90)]: Done 1340 tasks      | elapsed:   10.7s
[Parallel(n_jobs=90)]: Done 1890 tasks      | elapsed:   15.2s
[Parallel(n_jobs=90)]: Done 2540 tasks      | elapsed:   20.2s
[Parallel(n_jobs=90)]: Done 3290 tasks      | elapsed:   26.0s
[Parallel(n_jobs=90)]: Done 4140 tasks      | elapsed:   32.4s
[Parallel(n_jobs=90)]: Done 5090 tasks      | elapsed:   39.7s
[Parallel(n_jobs=90)]: Done 6270 out of 6270 | elapsed:   49.2s finished
/tmp/ipykernel_59243/2853696181.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  retinal_rod_markers = retinal_rod_markers.append(differential_exp_results, ignore_index=True)


OFF-bipolar cell


...

[Parallel(n_jobs=90)]: Using backend LokyBackend with 90 concurrent workers.
[Parallel(n_jobs=90)]: Done  20 tasks      | elapsed:    0.1s
[Parallel(n_jobs=90)]: Done 387 out of 566 | elapsed:    1.9s remaining:    0.9s
[Parallel(n_jobs=90)]: Done 566 out of 566 | elapsed:    5.0s finished
/tmp/ipykernel_59243/2853696181.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  retinal_rod_markers = retinal_rod_markers.append(differential_exp_results, ignore_index=True)


CD8-positive, alpha-beta T cell


[Parallel(n_jobs=90)]: Using backend LokyBackend with 90 concurrent workers.
[Parallel(n_jobs=90)]: Done  20 tasks      | elapsed:    0.2s
[Parallel(n_jobs=90)]: Done 360 tasks      | elapsed:    9.3s
[Parallel(n_jobs=90)]: Done 678 out of 857 | elapsed:   15.1s remaining:    4.0s
[Parallel(n_jobs=90)]: Done 857 out of 857 | elapsed:   21.0s finished
/tmp/ipykernel_59243/2853696181.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  retinal_rod_markers = retinal_rod_markers.append(differential_exp_results, ignore_index=True)


CD4-positive, alpha-beta T cell


[Parallel(n_jobs=90)]: Using backend LokyBackend with 90 concurrent workers.
[Parallel(n_jobs=90)]: Done  20 tasks      | elapsed:    0.2s
[Parallel(n_jobs=90)]: Done 368 tasks      | elapsed:    1.5s
[Parallel(n_jobs=90)]: Done 1061 tasks      | elapsed:    4.7s
[Parallel(n_jobs=90)]: Done 1687 tasks      | elapsed:    7.1s
[Parallel(n_jobs=90)]: Done 2248 tasks      | elapsed:    9.5s
[Parallel(n_jobs=90)]: Done 2274 out of 2453 | elapsed:    9.6s remaining:    0.8s
[Parallel(n_jobs=90)]: Done 2453 out of 2453 | elapsed:   12.6s finished
/tmp/ipykernel_59243/2853696181.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  retinal_rod_markers = retinal_rod_markers.append(differential_exp_results, ignore_index=True)


Now order genes based on quantile 0.1

In [9]:
retinal_rod_markers

,gene,de_coef,de_pval,de_se
0,RP4-669L17.4,0.195224,1.056152e-04,0.027794
1,LINC01409,0.023214,2.607479e-01,0.020120
2,SAMD11,2.328720,3.748820e-09,0.024257
3,NOC2L,-0.346713,1.998120e-04,0.018692
4,C1orf159,-0.669621,3.709827e-05,0.021069
...,...,...,...,...
320985,OTX2,1.176769,4.599080e-03,0.525220
320986,CTD-2368P22.1,0.093074,6.798640e-01,0.538497
320987,PCBP1-AS1,0.240299,6.366727e-01,0.665562
320988,MAB21L1,-1.138486,2.799440e-03,0.304808


In [23]:
retinal_rod_markers_summarized = retinal_rod_markers.groupby("gene").agg(lambda x: np.quantile(x, 0.1))

Top 25, they look well under Gene Expression and some of them are shared with the ones identified by Gene Expression

In [20]:
print(*list(retinal_rod_markers_summarized.sort_values("de_coef", ascending=False).head(25).index), sep = ", ")

RP11-35J10.5, PDE6A, EYS, RP11-171L9.1, SAG, USH2A, PDE6G, WFDC10B, NRL, RHO, RP11-103J8.1, RCVRN, ROM1, GNGT1, PCDH15, GNAT1, RP11-754N21.1, SLC24A1, HCN1, CNGA1, RP11-90C1.1, ARMC9, KCTD7, RP1, PDE6B


![](./eye.png)

## Find Marker Genes (SCRAN-inspired) for ALL cell types

Let's do all cell types vs each other cell type. First perform all comparions across other cell types.

In [ ]:
all_markers = {}
all_cell_types = adata.obs.cell_type.drop_duplicates()
total_cell_types = len(all_cell_types)
n = 1
for cell_type_query in all_cell_types:
    print("Current cell type", n, ":", cell_type_query)
    current_markers = pd.DataFrame()
    for cell_type_target in adata.obs.cell_type.drop_duplicates():

        if cell_type_target == cell_type_query:
            continue

        # Set groupings
        adata.obs[cell_type_target] = adata.obs.cell_type == cell_type_query
        adata.obs[cell_type_target] = adata.obs[cell_type_target] * 1

        # Run differential gene expression
        differential_exp_results = memento.binary_test_1d(
            adata=adata[adata.obs.cell_type.isin([cell_type_target, cell_type_query]),], 
            capture_rate=0.07, 
            treatment_col=cell_type_target, 
            num_cpus=90,
            num_boot=5000
        )

        # Retain columns of interest and rename them for future joining
        differential_exp_results = differential_exp_results[["gene", "de_coef", "de_pval", "de_se"]]

        if current_markers.shape == (0,0):
            current_markers = differential_exp_results
        else:
            current_markers = current_markers.append(differential_exp_results, ignore_index=True)
    
    # Calculate the 0.1 percentile for each gene across comparisons
    current_markers = current_markers.groupby("gene").agg(lambda x: np.quantile(x, 0.1))
    current_markers["cell_type"] = cell_type_query
    all_markers[cell_type_query] = current_markers

In [40]:
all_markers_consolidated = pd.concat(all_markers.values(), ignore_index = False )

In [59]:
all_markers_consolidated.sort_values("de_coef", ascending=False).head(25)
cell_type = 'retinal pigment epithelial cell'
top_25 = all_markers_consolidated.query(f"cell_type == '{cell_type}'").sort_values("de_coef", ascending=False).head(25)
print(*list(top_25.index), sep = ", ")

TTR, RPE65, SLC16A8, BEST1, RBP1, SLC22A8, PRDM16-DT, TMEM98, LINC00276, INPP5K, SERPINF1, SLC16A14, LRAT, SFRP5, LINC02159, RDH5, CTSD, ITGB8, SLC16A3, KCNJ13, PITPNA, CSPG5, PTGDS, IFITM10, TPRN


In [58]:
all_markers_consolidated.sort_values("de_coef", ascending=False).head(25)

,de_coef,de_pval,de_se,cell_type
gene,,,,
HBB,8.959019,5.812335e-10,0.092343,erythroid lineage cell
HBA2,8.554856,4.809925e-10,0.114799,erythroid lineage cell
HBA1,7.776886,1.687857e-08,0.124033,erythroid lineage cell
MUC5AC,6.848498,2.045492e-05,0.284006,goblet cell
DGAT2,5.930711,6.788103e-05,0.353516,fat cell
SCGB2A1,5.532811,3.970476e-08,0.193530,stromal cell
LACRT,5.506518,2.609889e-08,0.176430,stromal cell
RP11-554D15.3,5.460024,1.318281e-07,0.087348,lens fiber cell
RP11-434D9.1,5.353935,3.755857e-11,0.023825,retinal cone cell


In [27]:
import pickle
file_name = 'markers.pkl'

with open(file_name, 'wb') as file:
    pickle.dump(all_markers, file)

## Disorder code

In [ ]:
adata.obs.cell_type.isin([cell_type_target, cell_type_query])

In [ ]:
adata.var.loc[genes,cell_type_interest + "de_coef"] = list(differential_exp_results["de_coef"])
differential_exp_results["de_coef"]
adata.var.loc[genes, cell_type_interest + "de_coef"]

In [ ]:
adata.obs["dataset_id_cat"] = pd.Categorical(adata.obs.dataset_id)
adata.obs["dataset_id_cat"] = adata.obs["dataset_id_cat"].cat.codes

differential_exp_results = memento.binary_test_1d(
    adata=adata, 
    capture_rate=0.07, 
    treatment_col='differential_grouping', 
    num_cpus=90,
    num_boot=5000,
    covariate="dataset_id_cat")

In [ ]:
adata.var_names

In [60]:
census.close()